In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

import torchvision
import torchvision.transforms as transforms

In [6]:
torch.set_printoptions(linewidth=120)
train_set = torchvision.datasets.FashionMNIST(
root='./data/FashionMNIST', train=True, download=True,transform=transforms.Compose([
    transforms.ToTensor()
])) 

In [7]:
class MyNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):

        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        t = F.relu(self.fc1(t.reshape(-1, 12*4*4)))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        
        return t

In [10]:
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    for batch in loader:
        images, labels = batch
        
        preds = model(images)
        all_preds = torch.cat((all_preds, preds),dim=0)
        
    return all_preds
network = MyNetwork()

In [22]:
with torch.no_grad():
    prediction_loader = torch.utils.data.DataLoader(train_set, batch_size=10000)
    train_preds = get_all_preds(network, prediction_loader)

In [23]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

preds_correct = get_num_correct(train_preds, train_set.targets)

print('total correct:',preds_correct)
print('accuracy',preds_correct/len(train_set))

total correct: 6000
accuracy 0.1


In [24]:
train_set.targets

tensor([9, 0, 0,  ..., 3, 0, 5])

In [25]:
train_preds.argmax(dim=1)

tensor([1, 1, 1,  ..., 1, 1, 1])

In [27]:
stacked = torch.stack((train_set.targets, train_preds.argmax(dim=1)),dim=1)

In [28]:
stacked.shape

torch.Size([60000, 2])

In [29]:
stacked

tensor([[9, 1],
        [0, 1],
        [0, 1],
        ...,
        [3, 1],
        [0, 1],
        [5, 1]])

In [31]:
stacked[0].tolist()

[9, 1]

In [32]:
cmt = torch.zeros(10,10,dtype=torch.int64)
cmt

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [33]:
for p in stacked:
    j,k = p.tolist()
    cmt[j,k] +=1

In [38]:
cmt

tensor([[   0, 6000,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0, 6000,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0, 6000,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0, 6000,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0, 6000,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0, 6000,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0, 6000,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0, 6000,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0, 6000,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0, 6000,    0,    0,    0,    0,    0,    0,    0,    0]])